In [1]:
import os
import argparse
from distutils.version import LooseVersion
# Numerical libs
import numpy as np
import torch
import torch.nn as nn
from scipy.io import loadmat
import csv
# Our libs
from mit_semseg.dataset import TestDataset
from mit_semseg.models import ModelBuilder, SegmentationModule
from mit_semseg.utils import colorEncode, find_recursive, setup_logger
from mit_semseg.lib.nn import user_scattered_collate, async_copy_to
from mit_semseg.lib.utils import as_numpy
from PIL import Image
from tqdm import tqdm
from mit_semseg.config import cfg

In [2]:
colors = loadmat('../semantic-segmentation-pytorch/data/color150.mat')['colors']
print(colors)

[[120 120 120]
 [180 120 120]
 [  6 230 230]
 [ 80  50  50]
 [  4 200   3]
 [120 120  80]
 [140 140 140]
 [204   5 255]
 [230 230 230]
 [  4 250   7]
 [224   5 255]
 [235 255   7]
 [150   5  61]
 [120 120  70]
 [  8 255  51]
 [255   6  82]
 [143 255 140]
 [204 255   4]
 [255  51   7]
 [204  70   3]
 [  0 102 200]
 [ 61 230 250]
 [255   6  51]
 [ 11 102 255]
 [255   7  71]
 [255   9 224]
 [  9   7 230]
 [220 220 220]
 [255   9  92]
 [112   9 255]
 [  8 255 214]
 [  7 255 224]
 [255 184   6]
 [ 10 255  71]
 [255  41  10]
 [  7 255 255]
 [224 255   8]
 [102   8 255]
 [255  61   6]
 [255 194   7]
 [255 122   8]
 [  0 255  20]
 [255   8  41]
 [255   5 153]
 [  6  51 255]
 [235  12 255]
 [160 150  20]
 [  0 163 255]
 [140 140 140]
 [250  10  15]
 [ 20 255   0]
 [ 31 255   0]
 [255  31   0]
 [255 224   0]
 [153 255   0]
 [  0   0 255]
 [255  71   0]
 [  0 235 255]
 [  0 173 255]
 [ 31   0 255]
 [ 11 200 200]
 [255  82   0]
 [  0 255 245]
 [  0  61 255]
 [  0 255 112]
 [  0 255 133]
 [255   0 

In [3]:
names = {}
with open('../semantic-segmentation-pytorch/data/object150_info.csv') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        # only take the first label among the synonyms of the labels
        names[int(row[0])] = row[5].split(';')[0]
print(names)

{1: 'wall', 2: 'building', 3: 'sky', 4: 'floor', 5: 'tree', 6: 'ceiling', 7: 'road', 8: 'bed', 9: 'windowpane', 10: 'grass', 11: 'cabinet', 12: 'sidewalk', 13: 'person', 14: 'earth', 15: 'door', 16: 'table', 17: 'mountain', 18: 'plant', 19: 'curtain', 20: 'chair', 21: 'car', 22: 'water', 23: 'painting', 24: 'sofa', 25: 'shelf', 26: 'house', 27: 'sea', 28: 'mirror', 29: 'rug', 30: 'field', 31: 'armchair', 32: 'seat', 33: 'fence', 34: 'desk', 35: 'rock', 36: 'wardrobe', 37: 'lamp', 38: 'bathtub', 39: 'railing', 40: 'cushion', 41: 'base', 42: 'box', 43: 'column', 44: 'signboard', 45: 'chest', 46: 'counter', 47: 'sand', 48: 'sink', 49: 'skyscraper', 50: 'fireplace', 51: 'refrigerator', 52: 'grandstand', 53: 'path', 54: 'stairs', 55: 'runway', 56: 'case', 57: 'pool', 58: 'pillow', 59: 'screen', 60: 'stairway', 61: 'river', 62: 'bridge', 63: 'bookcase', 64: 'blind', 65: 'coffee', 66: 'toilet', 67: 'flower', 68: 'book', 69: 'hill', 70: 'bench', 71: 'countertop', 72: 'stove', 73: 'palm', 74: '

In [4]:
def visualize_result(data, pred, cfg):
    (img, info) = data

    # print predictions in descending order
    pred = np.int32(pred)
    pixs = pred.size
    uniques, counts = np.unique(pred, return_counts=True)
    print("Predictions in [{}]:".format(info))
    for idx in np.argsort(counts)[::-1]:
        name = names[uniques[idx] + 1]
        ratio = counts[idx] / pixs * 100
        if ratio > 0.1:
            print("  {}: {:.2f}%".format(name, ratio))

    # colorize prediction
    pred_color = colorEncode(pred, colors).astype(np.uint8)

    # aggregate images and save
    im_vis = np.concatenate((img, pred_color), axis=1)

    img_name = info.split('/')[-1]
    Image.fromarray(im_vis).save(
        os.path.join(cfg.TEST.result, img_name.replace('.jpg', '.png')))

In [5]:
def test(segmentation_module, loader, gpu):
    segmentation_module.eval()

    pbar = tqdm(total=len(loader))
    for batch_data in loader:
        # process data
        batch_data = batch_data[0]
        segSize = (batch_data['img_ori'].shape[0],
                   batch_data['img_ori'].shape[1])
        img_resized_list = batch_data['img_data']

        with torch.no_grad():
            scores = torch.zeros(1, cfg.DATASET.num_class, segSize[0], segSize[1])
            scores = async_copy_to(scores, gpu)

            for img in img_resized_list:
                feed_dict = batch_data.copy()
                feed_dict['img_data'] = img
                del feed_dict['img_ori']
                del feed_dict['info']
                feed_dict = async_copy_to(feed_dict, gpu)

                # forward pass
                pred_tmp = segmentation_module(feed_dict, segSize=segSize)
                scores = scores + pred_tmp / len(cfg.DATASET.imgSizes)

            _, pred = torch.max(scores, dim=1)
            pred = as_numpy(pred.squeeze(0).cpu())

        # visualization
        visualize_result(
            (batch_data['img_ori'], batch_data['info']),
            pred,
            cfg
        )

        pbar.update(1)    

In [6]:
assert LooseVersion(torch.__version__) >= LooseVersion('0.4.0'), \
        'PyTorch>=0.4.0 is required'

In [7]:
cfg_file = "config/ade20k-resnet50dilated-ppm_deepsup.yaml"
cfg.merge_from_file(cfg_file)
print(cfg)

DATASET:
  imgMaxSize: 1000
  imgSizes: (300, 375, 450, 525, 600)
  list_train: ../semantic-segmentation-pytorch/data/training.odgt
  list_val: ../semantic-segmentation-pytorch/data/validation.odgt
  num_class: 150
  padding_constant: 8
  random_flip: True
  root_dataset: ../semantic-segmentation-pytorch/data/
  segm_downsampling_rate: 8
DIR: model
MODEL:
  arch_decoder: ppm_deepsup
  arch_encoder: resnet50dilated
  fc_dim: 2048
  weights_decoder: 
  weights_encoder: 
TEST:
  batch_size: 1
  checkpoint: epoch_20.pth
  result: result
TRAIN:
  batch_size_per_gpu: 2
  beta1: 0.9
  deep_sup_scale: 0.4
  disp_iter: 20
  epoch_iters: 5000
  fix_bn: False
  lr_decoder: 0.02
  lr_encoder: 0.02
  lr_pow: 0.9
  num_epoch: 20
  optim: SGD
  seed: 304
  start_epoch: 0
  weight_decay: 0.0001
  workers: 16
VAL:
  batch_size: 1
  checkpoint: epoch_20.pth
  visualize: False


In [8]:
cfg.MODEL.arch_encoder = cfg.MODEL.arch_encoder.lower()
cfg.MODEL.arch_decoder = cfg.MODEL.arch_decoder.lower()
# absolute paths of model weights
cfg.MODEL.weights_encoder = os.path.join(cfg.DIR, 'encoder_' + cfg.TEST.checkpoint)
cfg.MODEL.weights_decoder = os.path.join(cfg.DIR, 'decoder_' + cfg.TEST.checkpoint)
# checkpoint does not exist at this point
#assert os.path.exists(cfg.MODEL.weights_encoder) and \
#        os.path.exists(cfg.MODEL.weights_decoder), "checkpoint does not exist!"

In [9]:
# create single joined image of left, center, and right images to test for inference
#img_names = ["/projects/ncdot/2018/NC_2018/327/19/327001906055.jpg", "/projects/ncdot/2018/NC_2018/327/19/327001906051.jpg", "/projects/ncdot/2018/NC_2018/327/19/327001906052.jpg"]
#for idx in range(3):
#    imgs[idx] = Image.open(img_names[idx])

#dst = Image.new('RGB', (imgs[0].width+imgs[1].width+imgs[2].width, imgs[0].height))
#dst.paste(imgs[0], (0,0))
#dst.paste(imgs[1], (imgs[0].width,0))
#dst.paste(imgs[2], (imgs[0].width+imgs[1].width,0))
#dst.save('image/32700190605_512.jpg')

In [10]:
#imgs = ["image/ADE_val_00001519.jpg"]
#imgs = ["image/32700190605_512.jpg"]
imgs = ["/projects/ncdot/2018/NC_2018/327/19/327001906052.jpg"]
#imgs = ["/projects/ncdot/2018/NC_2018/757/83/757012315276.jpg"]
#imgs = ["image/output0006.jpg"]
cfg.list_test = [{'fpath_img': x} for x in imgs]
print("cuda gpu device count:", torch.cuda.device_count())
gpu = torch.cuda.current_device()
print("cuda currently selected gpu device: ", gpu)
# torch.cuda.set_device(1)

cuda gpu device count: 2
cuda currently selected gpu device:  0


In [11]:
# Network Builders
net_encoder = ModelBuilder.build_encoder(
    arch=cfg.MODEL.arch_encoder,
    fc_dim=cfg.MODEL.fc_dim,
    weights=cfg.MODEL.weights_encoder)
net_decoder = ModelBuilder.build_decoder(
    arch=cfg.MODEL.arch_decoder,
    fc_dim=cfg.MODEL.fc_dim,
    num_class=cfg.DATASET.num_class,
    weights=cfg.MODEL.weights_decoder,
    use_softmax=True)

crit = nn.NLLLoss(ignore_index=-1)

segmentation_module = SegmentationModule(net_encoder, net_decoder, crit)

# Dataset and Loader
dataset_test = TestDataset(cfg.list_test, cfg.DATASET)
loader_test = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=cfg.TEST.batch_size,
    shuffle=False,
    collate_fn=user_scattered_collate,
    num_workers=5,
    drop_last=True)

segmentation_module.cuda()

# Main loop
test(segmentation_module, loader_test, gpu)

print('Inference done!')

Loading weights for net_encoder
Loading weights for net_decoder
# samples: 1


  0%|          | 0/1 [00:00<?, ?it/s]

Predictions in [/projects/ncdot/2018/NC_2018/327/19/327001906052.jpg]:
  sky: 29.10%
  tree: 27.74%
  grass: 21.61%
  earth: 11.24%
  fence: 5.24%
  road: 2.54%
  railing: 1.22%
  building: 0.69%
  path: 0.57%


100%|██████████| 1/1 [00:02<00:00,  2.43s/it]

Inference done!
